In [ ]:
using Pkg
using Revise
Pkg.activate("..") 
Pkg.instantiate()
using JPEC, Plots
gr() 

In [ ]:
using Revise
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

### Reading Eq

In [ ]:
# 1. Define the input parameters for the equilibrium solver.
#    - eq_filename: The name of the g-file we just created.
#    - eq_type: "efit" for a standard g-file.
#    - jac_type: "boozer" or "hamada" for the output coordinates.
#    - mpsi, mtheta: Resolution of the output grid.
    equil_control = JPEC.Equilibrium.EquilibriumControl(;
    eq_filename="beta_1.00",        # eq_filename
    eq_type="efit",          # eq_type
    jac_type="hamada",        # jac_type
    grid_type="ldp",
    psilow=0.01,             # psilow
    psihigh=0.990)             # psihigh
equil_config = JPEC.Equilibrium.EquilibriumConfig(equil_control,JPEC.Equilibrium.EquilibriumOutput())
# 2. Run the main equilibrium setup function.
#    This will read the file, solve the direct problem, and return the final object.
println("Starting equilibrium reconstruction...")
plasma_eq = JPEC.Equilibrium.setup_equilibrium(equil_config)
#plasma_eq = JPEC.Equilibrium.setup_equilibrium("./DIIID_example/equil.toml")
println("Equilibrium reconstruction complete.")

In [ ]:
fieldnames(typeof(plasma_eq.rzphi))

### CAlc. Metric

In [ ]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

In [ ]:
# Fourfit Metric Calculation Example
println("📚 Fourfit Metric Calculation")
println("="^30)

using .FourfitMetric

try
    # Metric tensor calculation
    metric_result = FourfitMetric.fourfit_make_metric(
        plasma_eq.rzphi,  # 2D geometry spline
        plasma_eq.sq;     # 1D profile spline
        mpsi=128,          # Radial grid points
        mtheta=256,        # Poloidal grid points  
        mband=10,          # Fourier bandwidth
        fft_flag=true,    # Use FFT
        verbose=true      # Detailed output
    )
    
    println("✅ Metric calculation complete!")
    println("   Grid size: $(size(metric_result.fs))")
    



    
    # Evaluate at a specific point (if spline is available)
    if metric_result.fspline !== nothing
        psi_test = 0.5
        theta_test = 0.5
        
        components = FourfitMetric.fspline_eval_metric(metric_result, psi_test, theta_test)
        println("   (ψ=$psi_test, θ=$theta_test):")
        println("   g11 = $(round(components[1], digits=6))")
        println("   g22 = $(round(components[2], digits=6))")
        println("   g33 = $(round(components[3], digits=6))")
        println("   g23 = $(round(components[4], digits=6))")
        println("   g31 = $(round(components[5], digits=6))")
        println("   g12 = $(round(components[6], digits=6))")

        println("   jac = $(round(components[7], digits=6))")
        println("   jac1 = $(round(components[8], digits=6))")

    end
    
catch e
    println("❌Error: $e")
end

### Calc. Metric2

In [ ]:
# Fourfit Metric Calculation Example
println("📚 Fourfit Metric Calculation")
println("="^30)

using .FourfitMetric

try
    # Metric tensor calculation
    metric_result = FourfitMetric.fourfit_make_metric(
        plasma_eq.rzphi,  # 2D geometry spline
        plasma_eq.sq;     # 1D profile spline
        mpsi=128,          # Radial grid points
        mtheta=256,        # Poloidal grid points  
        mband=20,          # Fourier bandwidth
        fft_flag=true,    # Use FFT
        verbose=true      # Detailed output
    )
    
    println("✅ Metric calculation complete!")
    println("   Grid size: $(size(metric_result.fs))")
    
    # 🔧 Define test coordinates FIRST
    psi_test = 0.5
    theta_test = 0*π/2
    
    println("\n🔍 Testing at coordinates: ψ=$psi_test, θ=$(round(theta_test/π, digits=2))π")

    # Test rzphi evaluation

        # Test rzphi evaluation
# Test rzphi evaluation
try
    rzphi_result = JPEC.SplinesMod.bicube_eval(plasma_eq.rzphi, psi_test, theta_test, 1)
    println("   rzphi values:")
    println("     f[1] (r²)     = $(round(rzphi_result[1], digits=6))")
    println("     f[2] (φ_shift)= $(round(rzphi_result[2], digits=6))")
    println("     f[3] (z_shift)= $(round(rzphi_result[3], digits=6))")
    println("     f[4] (jacobian)= $(round(rzphi_result[4], digits=6))")
    
    if length(rzphi_result) >= 8
        println("     fx[1] (∂r²/∂ψ) = $(round(rzphi_result[1+4], digits=6))")
        println("     fx[4] (∂J/∂ψ)  = $(round(rzphi_result[4+4], digits=6))")
    end
    
    if length(rzphi_result) >= 12
        println("     fy[1] (∂r²/∂θ) = $(round(rzphi_result[1+8], digits=6))")
        println("     fy[2] (∂φ/∂θ)  = $(round(rzphi_result[2+8], digits=6))")
    end
catch e
    println("   ❌ rzphi evaluation failed: $e")
end

# Test sq evaluation
try
    sq_result = JPEC.SplinesMod.spline_eval(plasma_eq.sq, psi_test, 1)
    println("   sq values:")
    println("     sq[1] (flux)   = $(round(sq_result[1], digits=6))")
    println("     sq[2] (pressure)= $(round(sq_result[2], digits=6))")
    println("     sq[3] (f=RBφ)  = $(round(sq_result[3], digits=6))")
    println("     sq[4] (q safety)= $(round(sq_result[4], digits=6))")
    
    if length(sq_result) >= 8
        println("     sq'[2] (p')    = $(round(sq_result[2+4], digits=6))")
        println("     sq'[4] (q')    = $(round(sq_result[4+4], digits=6))")
    end
catch e
    println("   ❌ sq evaluation failed: $e")
end



    # Test metric evaluation
    if metric_result.fspline !== nothing
        components = FourfitMetric.fspline_eval_metric(metric_result, psi_test, theta_test)
        println("   📈 Computed metric components:")
        println("     g11 = $(round(components[1], digits=6))")
        println("     g22 = $(round(components[2], digits=6))")
        println("     g33 = $(round(components[3], digits=6))")
        println("     g23 = $(round(components[4], digits=6))")
        println("     g31 = $(round(components[5], digits=6))")
        println("     g12 = $(round(components[6], digits=6))")
        println("     jac = $(round(components[7], digits=6))")
        println("     jac1 = $(round(components[8], digits=6))")
    end
    
catch e
    println("❌Error: $e")
end

In [ ]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

### Calc. Matric

In [ ]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

In [ ]:
using Revise

In [ ]:
using .FourfitMetric
using Interpolations

In [ ]:
    println("1️⃣  Calculating Metric Tensor...")
    metric_result = FourfitMetric.fourfit_make_metric(
        plasma_eq.rzphi,
        plasma_eq.sq;
        mpsi=128,
        mtheta=256,
        mband=10,
        fft_flag=false,
        verbose=true
    )
    

In [ ]:
metric_data=metric_result

In [ ]:
 mpsi = metric_data.mpsi          # psi 방향 격자점 수 (예: 128)
    mtheta = metric_data.mtheta      # theta 방향 격자점 수 (예: 256)  
    mband = metric_data.mband        # Fourier 모드 수 (예: 10)
    nqty = 8                         # metric 성분 수 (g11,g22,g33,g23,g31,g12,jac,jac1)
    
    # 원시 격자 데이터: fs[ipsi, itheta, iqty]
    fs_grid = metric_data.fs         # 이미 계산된 격자점 데이터
    n_theta = mtheta + 1             # 실제 theta 격자점 수
    

In [ ]:
using FFTW

In [ ]:
    coef = zeros(ComplexF64, mpsi + 1, mband + 1, nqty)
    
    for ipsi in 0:mpsi                # 각 psi 면에 대해
        for iqty in 1:nqty            # 각 metric 성분에 대해
            # 현재 psi 면의 theta 방향 데이터 추출
            theta_data = fs_grid[ipsi + 1, :, iqty]  # Julia 1-based indexing
            
            # FFT 수행 (theta 방향)
            fft_result = fft(theta_data)
            
            # Fourier 계수 추출 및 정규화
            for m in 0:mband
                if m == 0
                    # DC 성분 (m=0): fft_result의 첫 번째 원소
                    coef[ipsi + 1, m + 1, iqty] = fft_result[1] / n_theta
                else
                    # 양의 모드 (m > 0): fft_result의 (m+1) 번째 원소
                    if m + 1 <= length(fft_result) ÷ 2  # Nyquist 주파수 이하만
                        coef[ipsi + 1, m + 1, iqty] = fft_result[m + 1] / n_theta
                    else
                        coef[ipsi + 1, m + 1, iqty] = 0.0 + 0.0im
                    end
                end
            end
        end
    end

In [ ]:
    n_total_coeffs = nqty * (mband + 1)
    cs_fs = zeros(ComplexF64, mpsi + 1, n_total_coeffs)
    
    j = 0
    for iq in 1:nqty
        for m in 0:mband
            j += 1
            cs_fs[:, j] = coef[:, m + 1, iq]  # Julia 1-based 인덱싱
        end
    end

In [ ]:
size(cs_fs)

In [ ]:
function extract_coeffs_for_matrix_calculation_corrected(cs_fs, coef_3d, ipsi, mband)
    """
    cs_fs를 올바르게 사용하는 버전 (Fortran 방식)
    """
    # 로컬 배열 초기화
    g11, g22, g33, g23, g31, g12, jmat, jmat1 = 
        (zeros(ComplexF64, 2*mband + 1) for _ in 1:8)
    
    # Fortran의 layout_info에 따른 범위 정의
    g11_range = 1:(mband+1)
    g22_range = (mband+2):(2*mband+2)
    g33_range = (2*mband+3):(3*mband+3)
    g23_range = (3*mband+4):(4*mband+4)
    g31_range = (4*mband+5):(5*mband+5)
    g12_range = (5*mband+6):(6*mband+6)
    jmat_range = (6*mband+7):(7*mband+7)
    jmat1_range = (7*mband+8):(8*mband+8)
    
    # cs_fs에서 직접 계수 추출 (Fortran 방식)
    g11_coeffs = cs_fs[ipsi + 1, g11_range]      # m=0,1,2,...,mband
    g22_coeffs = cs_fs[ipsi + 1, g22_range]
    g33_coeffs = cs_fs[ipsi + 1, g33_range]
    g23_coeffs = cs_fs[ipsi + 1, g23_range]
    g31_coeffs = cs_fs[ipsi + 1, g31_range]
    g12_coeffs = cs_fs[ipsi + 1, g12_range]
    jmat_coeffs = cs_fs[ipsi + 1, jmat_range]
    jmat1_coeffs = cs_fs[ipsi + 1, jmat1_range]
    
    # dm = -mband:mband 범위로 배열 채우기
    for dm in -mband:mband
        dm_idx = dm + mband + 1        # Julia 배열 인덱스 (1 to 2*mband+1)
        m_abs = abs(dm)                # Fortran은 |m|만 저장
        
        if m_abs + 1 <= length(g11_coeffs)
            # 각 성분 할당
            g11[dm_idx] = g11_coeffs[m_abs + 1]
            g22[dm_idx] = g22_coeffs[m_abs + 1]
            g33[dm_idx] = g33_coeffs[m_abs + 1]
            g23[dm_idx] = g23_coeffs[m_abs + 1]
            g31[dm_idx] = g31_coeffs[m_abs + 1]
            g12[dm_idx] = g12_coeffs[m_abs + 1]
            jmat[dm_idx] = jmat_coeffs[m_abs + 1]
            jmat1[dm_idx] = jmat1_coeffs[m_abs + 1]
            
            # 음수 모드는 복소켤레 (Hermitian 성질)
            if dm < 0
                g11[dm_idx] = conj(g11[dm_idx])
                g22[dm_idx] = conj(g22[dm_idx])
                g33[dm_idx] = conj(g33[dm_idx])
                g23[dm_idx] = conj(g23[dm_idx])
                g31[dm_idx] = conj(g31[dm_idx])
                g12[dm_idx] = conj(g12[dm_idx])
                jmat[dm_idx] = conj(jmat[dm_idx])
                jmat1[dm_idx] = conj(jmat1[dm_idx])
            end
        end
    end
    
    return g11, g22, g33, g23, g31, g12, jmat, jmat1
end

In [ ]:
g11, g22, g33, g23, g31, g12, jmat, jmat1 = 
    extract_coeffs_for_matrix_calculation_corrected(cs_fs, coef, ipsi, mband)

In [ ]:
metric.fspline.fs[64,0,9]

In [ ]:
using LinearAlgebra

In [ ]:
# fourfit_make_matrix 함수 내에서:
verbose = true
sq = plasma_eq.sq
nn = 1  # 토로이달 모드 번호
mlow = -5  # 최소 폴로이달 모드모드
mhigh = 5  # 최대 폴로이달 모드모드
mpert = mhigh - mlow + 1  # 총 모드 수
twopi = 2π
ifac = 1.0im
psio = 2π  # 토로이달 자속 정규화

# --- 0. Fortran 방식으로 Fourier 계수 계산 ---
if verbose; println(">>> Computing Fourier coefficients (Fortran style)..."); end

# 행렬들 초기화
amats = zeros(ComplexF64, mpsi + 1, mpert^2)
bmats = zeros(ComplexF64, mpsi + 1, mpert^2)
cmats = zeros(ComplexF64, mpsi + 1, mpert^2)
dmats = zeros(ComplexF64, mpsi + 1, mpert^2)
emats = zeros(ComplexF64, mpsi + 1, mpert^2)
hmats = zeros(ComplexF64, mpsi + 1, mpert^2)
fmats = zeros(ComplexF64, mpsi + 1, mpert^2)
kmats = zeros(ComplexF64, mpsi + 1, mpert^2)

for ipsi in 0:mpsi
    psi_val = sq.xs[ipsi+1]
    
    # --- 2a. Define Flux Surface Quantities (Corrected) ---
    f_vals, f1_vals = JPEC.SplinesMod.spline_eval(sq, [psi_val], 1)
    profiles, profiles_d = f_vals[1, :], f1_vals[1, :]
    p1, q, q1, jtheta = profiles_d[2], profiles[4], profiles_d[4], -profiles_d[1]
    nq, chi1_psi = nn * q, twopi * psio
    
    # --- 2b. Extract Fourier Coefficients (Fortran 정확 방식) ---
    g11, g22, g33, g23, g31, g12, jmat, jmat1 = 
        extract_coeffs_for_matrix_calculation_corrected(cs_fs, coef, ipsi, mband)
    
    # imat 계수 (jacobian과 관련)
    imat = jmat  # 단순화, 실제로는 더 복잡할 수 있음
    
    # -----------------------------------------------------------------------
    # Begin loops over perturbed Fourier components
    # -----------------------------------------------------------------------
    ipert = 0
    amat = zeros(ComplexF64, mpert, mpert)
    bmat = zeros(ComplexF64, mpert, mpert)
    cmat = zeros(ComplexF64, mpert, mpert)
    dmat = zeros(ComplexF64, mpert, mpert)
    emat = zeros(ComplexF64, mpert, mpert)
    hmat = zeros(ComplexF64, mpert, mpert)
    fmat = zeros(ComplexF64, mpert, mpert)
    kmat = zeros(ComplexF64, mpert, mpert)
    
    for m1 in mlow:mhigh
        ipert += 1
        singfac1 = m1 - nq
        
        for dm in max(1 - ipert, -mband):min(mpert - ipert, mband)
            m2 = m1 + dm
            singfac2 = m2 - nq
            jpert = ipert + dm
            
            # 인덱스 범위 확인
            if jpert < 1 || jpert > mpert
                continue
            end
            
            dm_idx = dm + mband + 1  # Julia 인덱스 (1 to 2*mband+1)
            
            # -----------------------------------------------------------------------
            # Construct primitive matrices
            # -----------------------------------------------------------------------
            
            # A 행렬: 운동 에너지 항
            amat[ipert, jpert] = twopi^2 * (nn^2 * g22[dm_idx] +
                                            nn * (m1 + m2) * g23[dm_idx] +
                                            m1 * m2 * g33[dm_idx])
            
            # B 행렬: 혼합 항 1
            bmat[ipert, jpert] = -twopi * ifac * chi1_psi *
                                 (nn * g22[dm_idx] +
                                  (m1 + nq) * g23[dm_idx] +
                                  m1 * q * g33[dm_idx])
            
            # C 행렬: 복잡한 혼합 항
            cmat[ipert, jpert] = twopi * ifac *
                                 (twopi * ifac * chi1_psi * singfac2 *
                                  (nn * g12[dm_idx] + m1 * g31[dm_idx]) -
                                  q1 * chi1_psi * (nn * g23[dm_idx] + m1 * g33[dm_idx])) -
                                 twopi * ifac * (jtheta * singfac1 * imat[dm_idx] +
                                                 nn * p1 / chi1_psi * jmat[dm_idx])
            
            # D 행렬: 압력 구배 항
            dmat[ipert, jpert] = twopi * chi1_psi * (g23[dm_idx] + g33[dm_idx] * m1 / nn)
            
            # E 행렬: 혼합 항 2
            emat[ipert, jpert] = -chi1_psi / nn * (q1 * chi1_psi * g33[dm_idx] -
                                                   twopi * ifac * chi1_psi * g31[dm_idx] * singfac2 +
                                                   jtheta * imat[dm_idx])
            
            # H 행렬: 위치 에너지 항 (메인)
            hmat[ipert, jpert] = (q1 * chi1_psi)^2 * g33[dm_idx] +
                                 (twopi * chi1_psi)^2 * singfac1 * singfac2 * g11[dm_idx] -
                                 twopi * ifac * chi1_psi * dm * q1 * chi1_psi * g31[dm_idx] +
                                 jtheta * q1 * chi1_psi * imat[dm_idx] + p1 * jmat1[dm_idx]
            
            # F 행렬: 정규화 항
            fmat[ipert, jpert] = (chi1_psi / nn)^2 * g33[dm_idx]
            
            # K 행렬: 추가 혼합 항
            kmat[ipert, jpert] = twopi * ifac * chi1_psi * (g23[dm_idx] + g33[dm_idx] * m1 / nn)
        end
    end
    
    # -----------------------------------------------------------------------
    # Store matrices for this psi surface
    # -----------------------------------------------------------------------
    amats[ipsi+1, :] = reshape(amat, mpert^2)
    bmats[ipsi+1, :] = reshape(bmat, mpert^2)
    cmats[ipsi+1, :] = reshape(cmat, mpert^2)
    dmats[ipsi+1, :] = reshape(dmat, mpert^2)
    emats[ipsi+1, :] = reshape(emat, mpert^2)
    hmats[ipsi+1, :] = reshape(hmat, mpert^2)
    fmats[ipsi+1, :] = reshape(fmat, mpert^2)
    kmats[ipsi+1, :] = reshape(kmat, mpert^2)
    
    # -----------------------------------------------------------------------
    # Matrix factorization check
    # -----------------------------------------------------------------------
    try
        amat_fact = cholesky(Hermitian(amat), check=false)
        if !issuccess(amat_fact)
            if verbose
                println("⚠️  Warning: amat is near-singular at ipsi = $ipsi (psi = $(round(psi_val, digits=4)))")
            end
        end
    catch e
        if verbose
            println("❌ Factorization failed at ipsi = $ipsi: $e")
        end
    end
    
    # 중간 결과 출력 (디버깅용)
    if verbose && ipsi == mpsi ÷ 2
        println("   📊 Matrix check at ipsi = $ipsi (middle surface):")
        println("      amat condition: $(round(cond(amat), digits=2))")
        println("      amat determinant: $(round(abs(det(amat)), sigdigits=3))")
        println("      Sample amat[1,1]: $(round(amat[1,1], digits=6))")
        println("      Sample hmat[1,1]: $(round(hmat[1,1], digits=6))")
    end
end

if verbose
    println("✅ Matrix calculation completed for all $mpsi psi surfaces!")
    println("   Matrix dimensions: $mpert × $mpert")
    println("   Mode range: $mlow to $mhigh")
    println("   Stored matrices: A, B, C, D, E, H, F, K")
end

# -----------------------------------------------------------------------
# Create result structure
# -----------------------------------------------------------------------
matrix_result = (
    amats = amats,
    bmats = bmats, 
    cmats = cmats,
    dmats = dmats,
    emats = emats,
    hmats = hmats,
    fmats = fmats,
    kmats = kmats,
    mpert = mpert,
    mlow = mlow,
    mhigh = mhigh,
    mpsi = mpsi,
    xs = sq.xs,  # psi 좌표
    nn = nn,     # 토로이달 모드 번호
    mband = mband
)

println("🎉 All matrix calculations completed successfully!")

In [ ]:
println("🎉 All matrix calculations completed successfully!")

# -----------------------------------------------------------------------
# Initialize storage for compressed matrices
# -----------------------------------------------------------------------
if verbose; println(">>> Computing composite matrices and storage..."); end

# Hermitian 행렬 저장을 위한 크기 계산
hermitian_size = sum(min(mpert, jpert + mband) - jpert + 1 for jpert in 1:mpert)
kmat_size = sum(min(mpert, jpert + mband) - max(1, jpert - mband) + 1 for jpert in 1:mpert)

# 압축된 형태로 저장할 배열들
fmats_compressed = zeros(ComplexF64, mpsi + 1, hermitian_size)
gmats_compressed = zeros(ComplexF64, mpsi + 1, hermitian_size) 
kmats_compressed = zeros(ComplexF64, mpsi + 1, kmat_size)

# -----------------------------------------------------------------------
# Process each psi surface for composite matrices
# -----------------------------------------------------------------------
for ipsi in 0:mpsi
    psi_val = sq.xs[ipsi+1]
    
    # 기존에 계산된 행렬들 복원
    amat = reshape(amats[ipsi+1, :], mpert, mpert)
    bmat = reshape(bmats[ipsi+1, :], mpert, mpert)
    cmat = reshape(cmats[ipsi+1, :], mpert, mpert)
    dmat = reshape(dmats[ipsi+1, :], mpert, mpert)
    emat = reshape(emats[ipsi+1, :], mpert, mpert)
    hmat = reshape(hmats[ipsi+1, :], mpert, mpert)
    fmat = reshape(fmats[ipsi+1, :], mpert, mpert)
    kmat = reshape(kmats[ipsi+1, :], mpert, mpert)
    
    # -----------------------------------------------------------------------
    # 🔧 변수들을 미리 선언 (스코프 문제 해결)
    # -----------------------------------------------------------------------
    fmat_final = copy(fmat)  # 기본값으로 초기화
    kmat_final = copy(kmat)  # 기본값으로 초기화
    gmat_final = copy(hmat)  # 기본값으로 초기화
    
    # -----------------------------------------------------------------------
    # Compute composite matrices (Fortran 방식)
    # -----------------------------------------------------------------------
    temp1 = copy(dmat)
    temp2 = copy(cmat)
    
    try
        # A 행렬의 Hermitian 분해를 사용한 연립방정식 해결
        amat_herm = Hermitian(amat)
        amat_fact = cholesky(amat_herm, check=false)
        
        if issuccess(amat_fact)
            # A^(-1) * dmat와 A^(-1) * cmat 계산
            temp1 = amat_fact \ temp1  # A^(-1) * dmat
            temp2 = amat_fact \ temp2  # A^(-1) * cmat
            
            # 복합 행렬들 계산 (Fortran의 정확한 공식)
            fmat_final = fmat - adjoint(dmat) * temp1
            kmat_final = emat - adjoint(kmat) * temp2  
            gmat_final = hmat - adjoint(cmat) * temp2
            
        else
            if verbose
                println("⚠️  Warning: Using LU factorization for amat at ipsi = $ipsi")
            end
            # Cholesky가 실패하면 일반 LU 분해 사용
            temp1 = amat \ temp1
            temp2 = amat \ temp2
            
            fmat_final = fmat - adjoint(dmat) * temp1
            kmat_final = emat - adjoint(kmat) * temp2
            gmat_final = hmat - adjoint(cmat) * temp2
        end
        
    catch e
        if verbose
            println("❌ Matrix solve failed at ipsi = $ipsi: $e")
        end
        # 오류 시에는 이미 위에서 초기화된 값들을 사용
    end
    
    # -----------------------------------------------------------------------
    # Transfer F to banded matrix format (Hermitian)
    # -----------------------------------------------------------------------
    fmatb = zeros(ComplexF64, mband + 1, mpert)
    for jpert in 1:mpert
        for ipert in jpert:min(mpert, jpert + mband)
            band_idx = 1 + ipert - jpert
            if band_idx <= mband + 1
                fmatb[band_idx, jpert] = fmat_final[ipert, jpert]
            end
        end
    end
    
    # -----------------------------------------------------------------------
    # Factor F matrix (Cholesky for banded Hermitian)
    # -----------------------------------------------------------------------
    try
        fmat_herm = Hermitian(fmat_final)
        fmat_fact = cholesky(fmat_herm, check=false)
        if !issuccess(fmat_fact)
            if verbose && ipsi % 10 == 0
                println("⚠️  Warning: fmat is near-singular at ipsi = $ipsi")
            end
        end
    catch e
        if verbose && ipsi % 10 == 0
            println("❌ F-matrix factorization failed at ipsi = $ipsi: $e")
        end
    end
    
    # -----------------------------------------------------------------------
    # Store Hermitian matrices F and G (compressed format)
    # -----------------------------------------------------------------------
    iqty = 1
    for jpert in 1:mpert
        for ipert in jpert:min(mpert, jpert + mband)
            band_idx = 1 + ipert - jpert
            if iqty <= hermitian_size
                fmats_compressed[ipsi+1, iqty] = fmatb[band_idx, jpert]
                gmats_compressed[ipsi+1, iqty] = gmat_final[ipert, jpert]
                iqty += 1
            end
        end
    end
    
    # -----------------------------------------------------------------------
    # Store non-Hermitian matrix K
    # -----------------------------------------------------------------------
    iqty = 1
    for jpert in 1:mpert
        for ipert in max(1, jpert - mband):min(mpert, jpert + mband)
            if iqty <= kmat_size
                kmats_compressed[ipsi+1, iqty] = kmat_final[ipert, jpert]
                iqty += 1
            end
        end
    end
    
    # 진행상황 출력
    if verbose && ipsi % max(1, mpsi ÷ 10) == 0
        progress = round(100 * ipsi / mpsi, digits=1)
        println("   Composite matrix progress: $progress% (ipsi = $ipsi/$mpsi)")
        
        # 중간 점검 (중앙 면에서)
        if ipsi == mpsi ÷ 2
            println("   📊 Composite matrix check at ipsi = $ipsi:")
            println("      fmat condition: $(round(cond(fmat_final), digits=2))")
            println("      gmat condition: $(round(cond(gmat_final), digits=2))")
            println("      Sample fmat[1,1]: $(round(fmat_final[1,1], digits=6))")
            println("      Sample gmat[1,1]: $(round(gmat_final[1,1], digits=6))")
        end
    end
end

# -----------------------------------------------------------------------
# Create final result structure with compressed matrices
# -----------------------------------------------------------------------
matrix_result_final = (
    # 원본 primitive matrices (디버깅용)
    amats = amats,
    bmats = bmats, 
    cmats = cmats,
    dmats = dmats,
    emats = emats,
    hmats = hmats,
    fmats = fmats,  # 원본 F
    kmats = kmats,  # 원본 K
    
    # 압축된 composite matrices (실제 사용)
    fmats_compressed = fmats_compressed,  # F_final (compressed)
    gmats_compressed = gmats_compressed,  # G_final (compressed) 
    kmats_compressed = kmats_compressed,  # K_final (compressed)
    
    # 메타데이터
    mpert = mpert,
    mlow = mlow,
    mhigh = mhigh,
    mpsi = mpsi,
    mband = mband,
    xs = sq.xs,  # psi 좌표
    nn = nn,     # 토로이달 모드 번호
    hermitian_size = hermitian_size,
    kmat_size = kmat_size
)

if verbose
    println("✅ Composite matrix calculation completed!")
    println("   Final matrices: F ($(hermitian_size) elements), G ($(hermitian_size) elements), K ($(kmat_size) elements)")
    println("   Storage format: Hermitian compressed for F,G and banded for K")
    println("   Ready for eigenvalue analysis!")
end

println("🎉 All fourfit matrix calculations completed successfully!")

In [ ]:
# -----------------------------------------------------------------------
# 1️⃣ Spline fitting (ReadOnlyArray 문제 해결)
# -----------------------------------------------------------------------
println("4️⃣  Creating spline interpolators for matrices...")

# ReadOnlyArray를 일반 Vector로 변환
xs_vector = Vector{Float64}(sq.xs)  # 🔧 이 부분이 핵심!

# F, G, K 행렬을 위한 스플라인 생성 (수정된 방법)
fmats_spline = JPEC.SplinesMod.spline_setup(
    xs_vector,                # xs_nodes (일반 Vector로 변환)
    fmats_compressed;         # fs_nodes (2D array: [ipsi, iqty])
    bctype = 3                # boundary condition type (1=natural, 3=periodic)
)

gmats_spline = JPEC.SplinesMod.spline_setup(
    xs_vector,                # xs_nodes (일반 Vector로 변환)
    gmats_compressed;         # fs_nodes
    bctype = 3
)

kmats_spline = JPEC.SplinesMod.spline_setup(
    xs_vector,                # xs_nodes (일반 Vector로 변환)
    kmats_compressed;         # fs_nodes
    bctype = 3
)

# 2️⃣ Power setting (안정성 분석용)
power_flag = false  # 우선 기본값으로 설정

# 3️⃣ SAS (Small Aspect-ratio Stellarator) 모드 처리
sas_flag = false  # 우선 기본값으로 설정
psilim = 0.99     # 경계 위치

if sas_flag
    println("6️⃣  Processing SAS mode interpolation to psilim...")
    
    # SAS 모드를 위한 A, B, C 행렬 스플라인 생성 (여기도 xs_vector 사용)
    amats_spline = JPEC.SplinesMod.spline_setup(xs_vector, amats; bctype = 3)
    bmats_spline = JPEC.SplinesMod.spline_setup(xs_vector, bmats; bctype = 3)
    cmats_spline = JPEC.SplinesMod.spline_setup(xs_vector, cmats; bctype = 3)
    
    # psilim에서의 행렬 보간
    amat_limit, _, _, _ = JPEC.SplinesMod.spline_eval(amats_spline, [psilim], 0)
    bmat_limit, _, _, _ = JPEC.SplinesMod.spline_eval(bmats_spline, [psilim], 0)
    cmat_limit, _, _, _ = JPEC.SplinesMod.spline_eval(cmats_spline, [psilim], 0)
    
    # 경계 조건에서의 특별 처리
    amat_boundary = reshape(amat_limit[1, :], mpert, mpert)
    try
        amat_boundary_fact = cholesky(Hermitian(amat_boundary), check=false)
        if !issuccess(amat_boundary_fact)
            println("⚠️  Warning: Boundary amat is near-singular")
        end
    catch e
        println("❌ Boundary matrix factorization failed: $e")
    end
end

# 4️⃣ 진단 및 출력 옵션 (선택사항)
bin_metric = false
bin_fmat = false  
bin_gmat = false
bin_kmat = false

# -----------------------------------------------------------------------
# 최종 결과 구조체 업데이트 (수정된 버전)
# -----------------------------------------------------------------------
matrix_result_complete = (
    # 기존 데이터
    amats = amats,
    bmats = bmats, 
    cmats = cmats,
    dmats = dmats,
    emats = emats,
    hmats = hmats,
    fmats = fmats,
    kmats = kmats,
    
    # 압축된 composite matrices
    fmats_compressed = fmats_compressed,
    gmats_compressed = gmats_compressed, 
    kmats_compressed = kmats_compressed,
    
    # 🆕 스플라인 보간자들 (올바른 방법으로 생성)
    fmats_spline = fmats_spline,
    gmats_spline = gmats_spline,
    kmats_spline = kmats_spline,
    
    # 🆕 SAS 모드 데이터 (조건부)
    sas_flag = sas_flag,
    psilim = sas_flag ? psilim : nothing,
    amat_boundary = sas_flag ? amat_boundary : nothing,
    
    # 메타데이터
    mpert = mpert,
    mlow = mlow,
    mhigh = mhigh,
    mpsi = mpsi,
    mband = mband,
    xs = xs_vector,  # 🔧 여기도 변환된 벡터 사용
    nn = nn,
    hermitian_size = hermitian_size,
    kmat_size = kmat_size,
    power_flag = power_flag
)

println("✅ Complete fourfit matrix system ready!")
println("   📈 Spline interpolators: F, G, K matrices")
println("   🔧 Power factors: $(power_flag ? "enabled" : "disabled")")
println("   🎯 SAS mode: $(sas_flag ? "enabled" : "disabled")")
println("   🚀 Ready for eigenvalue analysis and stability studies!")
# -----------------------------------------------------------------------
# 5️⃣ 스플라인 사용 예제 (수정된 버전)
# -----------------------------------------------------------------------
println("\n🧪 Testing spline interpolation at arbitrary psi values...")

# 테스트할 psi 값들
test_psi_values = [0.3, 0.5, 0.7]

for psi_test in test_psi_values
    println("\n📍 Evaluating at ψ = $psi_test:")
    
    try
        # F, G, K 행렬을 해당 psi 값에서 평가 (수정된 방법)
        f_result = JPEC.SplinesMod.spline_eval(fmats_spline, [psi_test], 0)
        g_result = JPEC.SplinesMod.spline_eval(gmats_spline, [psi_test], 0)
        k_result = JPEC.SplinesMod.spline_eval(kmats_spline, [psi_test], 0)
        
        # 반환값 타입 확인 및 처리
        if isa(f_result, Tuple)
            # 튜플인 경우: (f, f1, f2, f3)
            f_interp, _, _, _ = f_result
            g_interp, _, _, _ = g_result
            k_interp, _, _, _ = k_result
        else
            # 단일 값인 경우
            f_interp = f_result
            g_interp = g_result
            k_interp = k_result
        end
        
        # 배열 형태 확인 및 처리
        if isa(f_interp, Array) && ndims(f_interp) >= 2
            # 2D 배열인 경우: [1, :]로 접근
            f_vec = f_interp[1, :]
            g_vec = g_interp[1, :]
            k_vec = k_interp[1, :]
        elseif isa(f_interp, Array) && ndims(f_interp) == 1
            # 1D 배열인 경우: 직접 사용
            f_vec = f_interp
            g_vec = g_interp
            k_vec = k_interp
        else
            # 스칼라인 경우: 벡터로 변환
            f_vec = [f_interp]
            g_vec = [g_interp]
            k_vec = [k_interp]
        end
        
        println("   ✅ F-matrix: $(length(f_vec)) elements")
        println("   ✅ G-matrix: $(length(g_vec)) elements")
        println("   ✅ K-matrix: $(length(k_vec)) elements")
        
        # 샘플 값 출력
        println("   📊 Sample values:")
        println("      F[1] = $(round(f_vec[1], digits=6))")
        println("      G[1] = $(round(g_vec[1], digits=6))")  
        println("      K[1] = $(round(k_vec[1], digits=6))")
        
        # 추가 디버깅 정보
        println("   🔍 Debug info:")
        println("      f_result type: $(typeof(f_result))")
        if isa(f_result, Tuple)
            println("      f_interp size: $(size(f_result[1]))")
        else
            println("      f_result size: $(size(f_result))")
        end
        
    catch e
        println("   ❌ Interpolation failed: $e")
        println("   🔍 Error details:")
        println("      Error type: $(typeof(e))")
        
        # 스플라인 구조 디버깅
        try
            println("      fmats_spline type: $(typeof(fmats_spline))")
            if hasfield(typeof(fmats_spline), :fs)
                println("      fmats_spline.fs size: $(size(fmats_spline.fs))")
            end
        catch debug_error
            println("      Spline debug failed: $debug_error")
        end
    end
end

# -----------------------------------------------------------------------
# 6️⃣ 대안적인 스플라인 평가 방법 시도
# -----------------------------------------------------------------------
println("\n🔄 Trying alternative spline evaluation methods...")

for psi_test in [0.5]  # 하나만 테스트
    println("\n📍 Alternative evaluation at ψ = $psi_test:")
    
    try
        # 방법 1: 다른 인수 순서 시도
        println("   Method 1: Different argument order")
        f_alt1 = JPEC.SplinesMod.spline_eval(fmats_spline, psi_test, 0)
        println("   Result: $(typeof(f_alt1)), value: $(f_alt1)")
        
    catch e1
        println("   Method 1 failed: $e1")
        
        try
            # 방법 2: 배열 없이 스칼라 전달
            println("   Method 2: Scalar input")
            f_alt2 = JPEC.SplinesMod.spline_eval(fmats_spline, psi_test)
            println("   Result: $(typeof(f_alt2)), value: $(f_alt2)")
            
        catch e2
            println("   Method 2 failed: $e2")
            
            try
                # 방법 3: fieldnames 확인
                println("   Method 3: Inspecting spline structure")
                spline_fields = fieldnames(typeof(fmats_spline))
                println("   Spline fields: $spline_fields")
                
                if :fs in spline_fields
                    println("   fs field size: $(size(fmats_spline.fs))")
                end
                
            catch e3
                println("   Method 3 failed: $e3")
            end
        end
    end
end

In [ ]:
sq.xs

In [ ]:
# fourfit_make_matrix 함수 내에서:
verbose=true
sq=plasma_eq.sq


# --- 0. Fortran 방식으로 Fourier 계수 계산 ---
if verbose; println(">>> Computing Fourier coefficients (Fortran style)..."); end

for ipsi in 0:mpsi
    psi_val = sq.xs[ipsi+1]
    

    # --- 2b. Extract Fourier Coefficients (Fortran 정확 방식) ---
    g11, g22, g33, g23, g31, g12, jmat, jmat1 = 
    extract_coeffs_for_matrix_calculation_corrected(cs_fs, coef, ipsi, mband)
    
    
    """# -----------------------------------------------------------------------
    # Begin loops over perturbed Fourier components
    # -----------------------------------------------------------------------
    ipert = 0
    amat, bmat, cmat, dmat, emat, hmat, fmat, kmat = (zeros(ComplexF64, mpert, mpert) for _ in 1:8)
    amats = zeros(ComplexF64, mpsi + 1, mpert^2)
    bmats = zeros(ComplexF64, mpsi + 1, mpert^2)
    cmats = zeros(ComplexF64, mpsi + 1, mpert^2)
    for m1 in mlow:mhigh
        ipert += 1
        singfac1 = m1 - nq
        for dm in max(1 - ipert, -mband):min(mpert - ipert, mband)
            m2 = m1 + dm
            singfac2 = m2 - nq
            jpert = ipert + dm

            # -----------------------------------------------------------------------
            # Construct primitive matrices
            # -----------------------------------------------------------------------
            amat[ipert, jpert] = twopi^2 * (nn^2 * g22[dm + mband + 1] +
                                            nn * (m1 + m2) * g23[dm + mband + 1] +
                                            m1 * m2 * g33[dm + mband + 1])
            bmat[ipert, jpert] = -twopi * ifac * chi1_psi *
                                 (nn * g22[dm + mband + 1] +
                                  (m1 + nq) * g23[dm + mband + 1] +
                                  m1 * q * g33[dm + mband + 1])
            cmat[ipert, jpert] = twopi * ifac *
                                 (twopi * ifac * chi1_psi * singfac2 *
                                  (nn * g12[dm + mband + 1] + m1 * g31[dm + mband + 1]) -
                                  q1 * chi1_psi * (nn * g23[dm + mband + 1] + m1 * g33[dm + mband + 1])) -
                                 twopi * ifac * (jtheta * singfac1 * imat[dm + mband + 1] +
                                                 nn * p1 / chi1_psi * jmat[dm + mband + 1])
            dmat[ipert, jpert] = twopi * chi1_psi * (g23[dm + mband + 1] + g33[dm + mband + 1] * m1 / nn)
            emat[ipert, jpert] = -chi1_psi / nn * (q1 * chi1_psi * g33[dm + mband + 1] -
                                                   twopi * ifac * chi1_psi * g31[dm + mband + 1] * singfac2 +
                                                   jtheta * imat[dm + mband + 1])
            hmat[ipert, jpert] = (q1 * chi1_psi)^2 * g33[dm + mband + 1] +
                                 (twopi * chi1_psi)^2 * singfac1 * singfac2 * g11[dm + mband + 1] -
                                 twopi * ifac * chi1_psi * dm * q1 * chi1_psi * g31[dm + mband + 1] +
                                 jtheta * q1 * chi1_psi * imat[dm + mband + 1] + p1 * jmat1[dm + mband + 1]
            fmat[ipert, jpert] = (chi1_psi / nn)^2 * g33[dm + mband + 1]
            kmat[ipert, jpert] = twopi * ifac * chi1_psi * (g23[dm + mband + 1] + g33[dm + mband + 1] * m1 / nn)
        end

        
    end
    if sas_flag
        amats[ipsi+1,:], bmats[ipsi+1,:], cmats[ipsi+1,:] = reshape(amat,mpert^2), reshape(bmat,mpert^2), reshape(cmat,mpert^2)
    end

    amat_fact = cholesky(Hermitian(amat), check=false)
    if !issuccess(amat_fact)
        error("Factorization failed: amat is singular at ipsi = $ipsi.")
    end
    if ipsi==mpsi/2
        println("Final amat matrix at ipsi = $ipsi:")
        println(amats[:,:])
    end
    # 이제 나머지 행렬 계산 계속..."""
end

In [ ]:
g11[11]

In [ ]:
    g11,g22,g33,g23,g31,g12,jmat,jmat1 = (zeros(ComplexF64, 2*mband + 1) for _ in 1:8)

for ipsi in 0:mpsi

        psi_val = sq.xs[ipsi+1]

        # --- 2a. Define Flux Surface Quantities (Corrected) ---
        f_vals, f1_vals = JPEC.SplinesMod.spline_eval(sq, [psi_val], 1)
        profiles, profiles_d = f_vals[1, :], f1_vals[1, :]
        p1, q, q1, jtheta = profiles_d[2], profiles[4], profiles_d[4], -profiles_d[1]
        nq, chi1_psi = nn * q, twopi * psio
        for dm in -mband:mband
            dm_idx = dm + mband + 1 # Index for our local g11, g22, etc. arrays (1 to 2*mband+1)
            g11[dm_idx]   = metric_coeffs[ipsi+1, 1 + dm_idx - 1] #/ n_theta
            g22[dm_idx]   = metric_coeffs[ipsi+1, 2 + dm_idx - 1] #/ n_theta
            g33[dm_idx]   = metric_coeffs[ipsi+1, 3 + dm_idx - 1] #/ n_theta
            g23[dm_idx]   = metric_coeffs[ipsi+1, 4 + dm_idx - 1] #/ n_theta
            g31[dm_idx]   = metric_coeffs[ipsi+1, 5 + dm_idx - 1] #/ n_theta
            g12[dm_idx]   = metric_coeffs[ipsi+1, 6 + dm_idx - 1] #/ n_theta
            jmat[dm_idx]  = metric_coeffs[ipsi+1, 7 + dm_idx - 1] #/ n_theta
            jmat1[dm_idx] = metric_coeffs[ipsi+1, 8 + dm_idx - 1] #/ n_theta
        end

    end

In [ ]:
jmat[11]

In [ ]:
println(size(metric_coeffs[1,:]))


In [ ]:
using FFTW


In [ ]:
g11[3]

In [ ]:
        fft_coeffs = fft(theta_slice, 1)


In [ ]:
metric_result.fspline.fs

In [ ]:
println("checking")

In [ ]:
# Fourfit Matrix Calculation Example
println("🚀 Fourfit Matrix Calculation")
println("="^30)

using .FourfitMetric
using Interpolations

try
    # Step 1: Metric tensor calculation
    println("1️⃣  Calculating Metric Tensor...")
    metric_result = FourfitMetric.fourfit_make_metric(
        plasma_eq.rzphi,
        plasma_eq.sq;
        mpsi=128,
        mtheta=256,
        mband=10,
        fft_flag=false,
        verbose=true
    )
    
    # Step 2: Matrix calculation
    println("2️⃣  Calculating MHD Coefficient Matrix...")
    
    # Calculate psio (Toroidal flux normalization)
    psio = 2π  # Default value, should be obtained from equilibrium data

    matrix_result = FourfitMetric.fourfit_make_matrix(
        metric_result,
        plasma_eq.sq,
        plasma_eq.rzphi,
        psio;
        nn=1,           # Toroidal mode number
        mlow=-5,        # Minimum poloidal mode
        mhigh=5,        # Maximum poloidal mode
        power_flag=false,
        feval_flag=true,
        sas_flag=true
    )
    
    # Step 3: Analyze results
    println("3️⃣  Analyzing Results:")
    println("   Matrix dimensions: $(size(matrix_result.amat))")
    println("   Mode range: $(matrix_result.mlow) to $(matrix_result.mhigh)")
    println("   Number of modes: $(matrix_result.mpert)")
    
    # Example of eigenvalue computation
    if matrix_result.amat !== nothing
        eigenvals = FourfitMetric.compute_eigenvalues(matrix_result, 0.5)
        if length(eigenvals) > 0
            println("   First 3 eigenvalues:")
            for (i, λ) in enumerate(eigenvals[1:min(3, length(eigenvals))])
                println("     λ[$i] = $(round(real(λ), digits=6)) + $(round(imag(λ), digits=6))i")
            end
        end
    end
    
    println("✅ All calculations completed successfully!")
    
catch e
    println("❌ Error occurred: $e")
    println("\n🔍 Error Analysis:")
    if isa(e, UndefVarError)
        println("   Undefined variable error: $(e.var)")
        println("   This might be a module scope issue.")
    elseif isa(e, MethodError)
        println("   Method error: $(e.f)")
        println("   Argument types: $(typeof.(e.args))")
        println("   This might be an issue with spline evaluation.")
    elseif isa(e, BoundsError)
        println("   Array index out of bounds error")
        println("   Please check the array dimensions.")
    end
    println("\n💡 Suggestions:")
    println("   1. Ensure equilibrium data is loaded correctly.")
    println("   2. Check the return format of spline_eval.")
    println("   3. Verify all variables are defined in the correct scope.")
end


In [ ]:
fieldnames(typeof(matrix_result.fmats.fs))

In [ ]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues